# Category Theory for Programmers

## Chapter 2: Types and Functions

### Challenge 5
There are $|\text{Bool}|^{|\text{Bool}|^1} = 2^2 = 4$ functions from `Bool` to `Bool`.

In [ ]:
f1 :: Bool -> Bool
f1 = id

f2 :: Bool -> Bool
f2 = not

f3 :: Bool -> Bool
f3 = const False

f4 :: Bool -> Bool
f4 = const True

## Chapter 3: Categories Great and Small

### Challenge 3

```haskell
import Data.Monoid

instance Monoid Bool where
    mempty = True
    mappend = (&&)

instance Monoid Bool where
    mempty = False
    mappend = (||)
```

### Challenge 5

In [ ]:
import Data.Monoid
import Data.Semigroup

newtype Z3 = Z3 Int
    deriving Show

instance Semigroup Z3 where
    (Z3 x) <> (Z3 y) = Z3 ((x + y) `mod` 3)

instance Monoid Z3 where
    mempty = Z3 0

In [ ]:
foldMap Z3 [2, 1, 2]

## Chapter 6: Simple Algebraic Data Types

### Challenge 1
Show the isomorphism between `Maybe a` and `Either () a`.

In [ ]:
maybeToEither :: Maybe a -> Either () a
maybeToEither Nothing = Left ()
maybeToEither (Just x) = Right x

eitherToMaybe :: Either () a -> Maybe a
eitherToMaybe (Left _) = Nothing
eitherToMaybe (Right x) = Just x

In [ ]:
eitherToMaybe $ maybeToEither Nothing

In [ ]:
eitherToMaybe $ maybeToEither $ Just 42

### Challenge 4
Define new shape `Square` and extend `area` and `circ`.

In [ ]:
data Shape = Circle Float | Rect Float Float | Square Float

area :: Shape -> Float
area (Circle r) = pi * r^2
area (Rect d h) = d * h
area (Square s) = s^2

circ :: Shape -> Float
circ (Circle r) = 2 * pi * r
circ (Rect d h) = 2 * (d + h)
circ (Square s) = 4 * s

### Challenge 5
Show that $a + a = 2 \times a$ up to isomorphism where $a \times a$ is `(a, b)` and $a + b$ is `Either a b` ($2$ in this setup corresponds to `Bool`).

The definition below works up to isomorphism in the choice between
 1. `Left -> False` vs `Left -> True`
 1. `(Bool, a)` vs `(a, Bool)`

In [ ]:
toFlagged :: Either a a -> (Bool, a)
toFlagged (Left x) = (False, x)
toFlagged (Right x) = (True, x)

fromFlagged :: (Bool, a) -> Either a a
fromFlagged (False, x) = Left x
fromFlagged (True, x) = Right x